In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn import mixture
from sklearn import metrics
%matplotlib inline

In [17]:
def prepareTrainSet():    
    df = pd.read_csv('labeled_sina_new.csv')
    df = df[['orderid','IsSpoof','cancelled buy','exec sell','cancelled sell','exec buy','microsecond','price','side','time','date','inventory','time diff',
         'ewav_back canc buy','ewav_back canc sell','ewav_back exec buy','ewav_back exec sell','ewav_back buy/sell','ewav_back sell/buy']]
    del df['ewav_back exec buy']
    del df['ewav_back exec sell']
    # clean the data for ewav_back canc buy/sell and sell/buy
    # buy/sell will be just inverse of sell/buy, so we use one column buy/sell
    df.loc[(df['ewav_back canc buy']==0)&(df['ewav_back canc sell']==0),'ewav_back buy/sell']=1
    medianbs = df.loc[(df['ewav_back buy/sell']>0)&(df['ewav_back buy/sell']<np.inf),'ewav_back buy/sell'].median()
    maxbs = df.loc[(df['ewav_back buy/sell']>0)&(df['ewav_back buy/sell']<np.inf),'ewav_back buy/sell'].max()
    df.loc[df['ewav_back buy/sell']==np.inf,'ewav_back buy/sell'] = maxbs
    df.loc[df['ewav_back buy/sell']==0,'ewav_back buy/sell'] = 1/maxbs
    df['ewav_back buy/sell'] = df['ewav_back buy/sell'].map(np.log)
    # To get the time difference between current trade and its nearest trade at the same side
    df['TimeDiff_back'] = np.nan
    df['TimeDiff_frwd'] = np.nan
    df['TimeDiff_min'] = np.nan
    df = df.loc[(df['exec sell']>0)|(df['exec buy']>0),:].copy()

    buy = df.loc[df['side']=='B',:].copy()
    for dd in buy['date'].unique():
        #import pdb;pdb.set_trace()
        tmp = buy.loc[buy['date']==dd,:]
        buy.loc[buy['date']==dd,'TimeDiff_back'] = buy.loc[buy['date']==dd,'microsecond'].diff(1).map(lambda x:np.abs(x))
        buy.loc[buy['date']==dd,'TimeDiff_frwd'] = buy.loc[buy['date']==dd,'microsecond'].diff(-1).map(lambda x:np.abs(x))
    #import pdb;pdb.set_trace()    
    buy['TimeDiff_frwd'] = buy['TimeDiff_frwd'].fillna(buy['TimeDiff_frwd'].max())    
    buy['TimeDiff_back'] = buy['TimeDiff_back'].fillna(buy['TimeDiff_back'].max())
    buy['TimeDiff_min'] = buy.apply(lambda x:min(x['TimeDiff_back'],x['TimeDiff_frwd']),axis=1)

    sell = df.loc[df['side']=='S',:].copy()
    for dd in sell['date'].unique():
        tmp = sell.loc[sell['date']==dd,:]
        sell.loc[sell['date']==dd,'TimeDiff_back'] = sell.loc[sell['date']==dd,'microsecond'].diff(1).map(lambda x:np.abs(x))
        sell.loc[sell['date']==dd,'TimeDiff_frwd'] = sell.loc[sell['date']==dd,'microsecond'].diff(-1).map(lambda x:np.abs(x))
    
    sell['TimeDiff_frwd'] = sell['TimeDiff_frwd'].fillna(sell['TimeDiff_frwd'].max())
    sell['TimeDiff_back'] = sell['TimeDiff_back'].fillna(sell['TimeDiff_back'].max())
    sell['TimeDiff_min'] = sell.apply(lambda x:min(x['TimeDiff_back'],x['TimeDiff_frwd']),axis=1)

    newdf = buy.append(sell)
    newdf['date'] = newdf['date'].map(lambda x:pd.to_datetime(x))
    #newdf = newdf.sort(['date','microsecond'])
    df = newdf.sort()
    #Define states
    s1 = {'side':'B','IsSpoof':False}
    s2 = {'side':'S','IsSpoof':False}
    s3 = {'side':'B','IsSpoof':True}
    s4 = {'side':'S','IsSpoof':True}

    df['state']=0
    df.loc[(df['side']=='B')&(df['IsSpoof']==False),'state'] = 0
    df.loc[(df['side']=='S')&(df['IsSpoof']==False),'state'] = 1
    df.loc[(df['side']=='B')&(df['IsSpoof']==True),'state'] = 2
    df.loc[(df['side']=='S')&(df['IsSpoof']==True),'state'] = 3
    
    return df

In [10]:
def toMS(x):
    return ((x.hour*60+x.minute)*60+x.second)*1000000+x.microsecond
    
    
class DataPrep:
    def __init__(self):
        pass
    
    def processData(self,filename):
        data = pd.read_csv(filename)
        print '----Data cleaning----'
        
        allorder = self.cleanData(data)
        data = self.prepare(allorder)
        
        print '---- Feature calculation----'
        #data = self.computeEWAVForward(data)
        allorder = self.computeEWAVBackward(data)
        #data = self.computeSEV(data)
        
        print '----- Prepare for HMM------'
        data = self.HMMPrep(allorder.copy())
        return allorder,data
    
    def computeEWAVBackward(self,data):
        if len(data)<2:
            raise ValueError('data too short')
        T=np.median(data['time diff'])*2
        data['ewav_back canc buy']=0
        data['ewav_back canc sell']=0
        data['ewav_back exec buy']=0
        data['ewav_back exec sell']=0
    
        for ii in range(1,len(data)):
            data.loc[ii,'ewav_back canc buy'] = data.loc[ii,'cancelled buy']+data.loc[ii-1,'ewav_back canc buy']*math.exp(-data.ix[ii]['time diff']/T)
            data.loc[ii,'ewav_back canc sell'] = data.loc[ii,'cancelled sell']+data.loc[ii-1,'ewav_back canc sell']*math.exp(-data.ix[ii]['time diff']/T)
            data.loc[ii,'ewav_back exec buy'] = data.loc[ii,'exec buy']+data.loc[ii-1,'ewav_back exec buy']*math.exp(-data.ix[ii]['time diff']/T)
            data.loc[ii,'ewav_back exec sell'] = data.loc[ii,'exec sell']+data.loc[ii-1,'ewav_back exec sell']*math.exp(-data.ix[ii]['time diff']/T)
        data['ewav_back buy/sell'] = 0
        data.loc[data['ewav_back canc sell']==0,'ewav_back buy/sell']= np.inf
        data.loc[data['ewav_back canc sell']!=0,'ewav_back buy/sell'] = data.loc[data['ewav_back canc sell']!=0,'ewav_back canc buy']/data.loc[data['ewav_back canc sell']!=0,'ewav_back canc sell']
    
        data['ewav_back sell/buy'] = 0
        data.loc[data['ewav_back canc buy']==0,'ewav_back sell/buy'] = np.inf
        data.loc[data['ewav_back canc buy']!=0,'ewav_back sell/buy'] = data.loc[data['ewav_back canc buy']!=0,'ewav_back canc sell']/data.loc[data['ewav_back canc buy']!=0,'ewav_back canc buy']
    
        data['ewav_back buy exec/total'] = 0
        data['ewav_back buy exec+canc'] = data['ewav_back exec buy'] + data['ewav_back canc buy']
        data.loc[data['ewav_back buy exec+canc']==0,'ewav_back buy exec/total']= 1
        data.loc[data['ewav_back buy exec+canc']!=0,'ewav_back buy exec/total'] = data.loc[data['ewav_back buy exec+canc']!=0,'ewav_back exec buy']/data.loc[data['ewav_back buy exec+canc']!=0,'ewav_back buy exec+canc']
    
        data['ewav_back sell exec/total'] = 0
        data['ewav_back sell exec+canc'] = data['ewav_back exec sell'] + data['ewav_back canc sell']
        data.loc[data['ewav_back sell exec+canc']==0,'ewav_back sell exec/total'] = 1
        data.loc[data['ewav_back sell exec+canc']!=0,'ewav_back sell exec/total'] = data.loc[data['ewav_back sell exec+canc']!=0,'ewav_back exec sell']/data.loc[data['ewav_back sell exec+canc']!=0,'ewav_back sell exec+canc']
    
        return data
    def computeEWAVForward(self,data):
        if len(data)<2:
            raise ValueError('data too short')
        T=np.median(data['time diff'])*2
        data['ewav_for canc buy']=0
        data['ewav_for canc sell']=0
        data['ewav_for exec buy']=0
        data['ewav_for exec sell']=0
    
        for ii in range(len(data)-2,-1,-1):
            data.loc[ii,'ewav_for canc buy'] = data.loc[ii,'cancelled buy']+data.loc[ii+1,'ewav_for canc buy']*math.exp(-data.ix[ii+1]['time diff']/T)
            data.loc[ii,'ewav_for canc sell'] = data.loc[ii,'cancelled sell']+data.loc[ii+1,'ewav_for canc sell']*math.exp(-data.ix[ii+1]['time diff']/T)
            data.loc[ii,'ewav_for exec buy'] = data.loc[ii,'exec buy']+data.loc[ii+1,'ewav_for exec buy']*math.exp(-data.ix[ii+1]['time diff']/T)
            data.loc[ii,'ewav_for exec sell'] = data.loc[ii,'exec sell']+data.loc[ii+1,'ewav_for exec sell']*math.exp(-data.ix[ii+1]['time diff']/T)
        data['ewav_for buy/sell'] = 0
        data.loc[data['ewav_for canc sell']==0,'ewav_for buy/sell']= np.inf
        data.loc[data['ewav_for canc sell']!=0,'ewav_for buy/sell'] = data.loc[data['ewav_for canc sell']!=0,'ewav_for canc buy']/data.loc[data['ewav_for canc sell']!=0,'ewav_for canc sell']
    
        data['ewav_for sell/buy'] = 0
        data.loc[data['ewav_for canc buy']==0,'ewav_for sell/buy'] = np.inf
        data.loc[data['ewav_for canc buy']!=0,'ewav_for sell/buy'] = data.loc[data['ewav_for canc buy']!=0,'ewav_for canc sell']/data.loc[data['ewav_for canc buy']!=0,'ewav_for canc buy']
    
        data['ewav_for buy exec/total'] = 0
        data['ewav_for buy exec+canc'] = data['ewav_for exec buy'] + data['ewav_for canc buy']
        data.loc[data['ewav_for buy exec+canc']==0,'ewav_for buy exec/total']= 1
        data.loc[data['ewav_for buy exec+canc']!=0,'ewav_for buy exec/total'] = data.loc[data['ewav_for buy exec+canc']!=0,'ewav_for exec buy']/data.loc[data['ewav_for buy exec+canc']!=0,'ewav_for buy exec+canc']
    
        data['ewav_for sell exec/total'] = 0
        data['ewav_for sell exec+canc'] = data['ewav_for exec sell'] + data['ewav_for canc sell']
        data.loc[data['ewav_for sell exec+canc']==0,'ewav_for sell exec/total'] = 1
        data.loc[data['ewav_for sell exec+canc']!=0,'ewav_for sell exec/total'] = data.loc[data['ewav_for sell exec+canc']!=0,'ewav_for exec sell']/data.loc[data['ewav_for sell exec+canc']!=0,'ewav_for sell exec+canc']
    
        return data
    
    def computeSEV(self,data):
        data['sev buy']=0
        data['sev sell']=0
    
        winsize = 4*np.median(data['time diff'])
        top=len(data)-1
        tail = top
        while(top>=0):
            while tail>=top:
                if data.loc[tail,'microsecond']-data.loc[top,'microsecond'] <=winsize:
                    break
                tail-=1
            tmpbuy = 0.
            tmpsell = 0.
            for ii in range(top,tail+1):
                tmpbuy += data.loc[ii,'exec buy']
                tmpsell += data.loc[ii,'exec sell']
            data.loc[top,'sev buy']=tmpbuy
            data.loc[top,'sev sell']=tmpsell
            top -= 1
        data['sev net buy'] = data['sev buy'] - data['sev sell']
        data.loc[data['sev buy']==0,'sev sell/buy'] = np.inf
        data.loc[data['sev buy']!=0,'sev sell/buy'] = data.loc[data['sev buy']!=0,'sev sell']/data.loc[data['sev buy']!=0,'sev buy']*1.
        data.loc[data['sev sell']==0,'sev buy/sell'] = np.inf
        data.loc[data['sev sell']!=0,'sev buy/sell'] = data.loc[data['sev sell']!=0,'sev buy']/data.loc[data['sev sell']!=0,'sev sell']*1.
        return data  
    
    def cleanData(self,data):
        
        data['execution_time'] = data['execution_time'].map(lambda x:pd.to_datetime(x))
        data['cancel_entry_time'] = data['cancel_entry_time'].map(lambda x:pd.to_datetime(x))
        data['order_entry_time'] = data['order_entry_time'].map(lambda x:pd.to_datetime(x))
        data['prc*qty'] = data['q_exec']*data['prc_exec']

        neworder = data.loc[data['order_type']=='NEW ORDER',:]
        exeorder = data.loc[data['order_type']=='EXECUTION',:]
        canorder = data.loc[data['order_type']=='CANCEL',:].copy()
    
        ############## Exclude those partial filled orders from cancel list
        #partialfill = set(canorder['orderid']).intersection(set(exeorder['orderid']))
        #canorder = canorder.loc[canorder['orderid'].isin(partialfill)==False,:]
        #####################################################################
   
        allorder = neworder[['id','orderid','symbol','q_new','price','order_entry_time','date','time','side']].set_index('orderid')
        gp = exeorder.groupby('orderid')
        tmp = gp.agg({'q_exec':np.sum,'prc*qty':np.sum})
        tmp['avg exe_prc'] = tmp['prc*qty']/tmp['q_exec']
        del tmp['prc*qty']
        allorder = allorder.join(tmp)
    #allorder = allorder.join(gp['execute_time'].agg({'first_exe_time':np.min,'last_exe_time':np.max}))
        allorder = allorder.join(gp['execution_time'].agg({'first_execution_time':np.min,'last_execution_time':np.max}))
        allorder['execution_time_first_ms'] = allorder['first_execution_time'].map(toMS)
        allorder['execution_time_last_ms'] = allorder['last_execution_time'].map(toMS)
        allorder['order_entry_time_ms'] = allorder['order_entry_time'].map(toMS)
    #gp = canorder.groupby('orderid')
        allorder = allorder.join(canorder.set_index('orderid')[['cancel_entry_time','canc_time']])
        allorder['q_exec'].fillna(0,inplace=True)
        allorder['q_cancel'] = allorder['q_new'] - allorder['q_exec']
        allorder = allorder.sort('order_entry_time')
        return allorder
    
    def prepare(self,allorder):
    #import pdb;pdb.set_trace()
        fillorder = allorder.loc[allorder['q_exec']>0,['date','price','side','last_execution_time','execution_time_last_ms','q_exec']]
        fillorder['exec buy'] = fillorder['q_exec']
        fillorder['exec sell'] = fillorder['q_exec']
        fillorder.loc[fillorder['side']=='B','exec sell'] = 0
        fillorder.loc[fillorder['side']!='B','exec buy'] = 0
        fillorder = fillorder.rename(columns={'execution_time_last_ms':'microsecond','last_execution_time':'time'})

    #canorder = allorder.loc[allorder['q_cancel']>0,['date','price','side','order_entry_time','order_entry_time_ms','q_cancel']]
        canorder = allorder.loc[allorder['q_cancel']==allorder['q_new'],['date','price','side','order_entry_time','order_entry_time_ms','q_cancel']]
    #partially filled order discarded
    #import pdb;pdb.set_trace()
        canorder['cancelled buy'] = canorder['q_cancel']
        canorder['cancelled sell'] = canorder['q_cancel']
        canorder.loc[canorder['side']=='B','cancelled sell'] = 0.0
        canorder.loc[canorder['side']!='B','cancelled buy'] = 0.0
        canorder = canorder.rename(columns={'order_entry_time_ms':'microsecond','order_entry_time':'time'})

        fillorder['cancelled buy'] = 0
        fillorder['cancelled sell'] = 0
        canorder['exec buy'] = 0
        canorder['exec sell'] = 0
        del canorder['q_cancel']
        del fillorder['q_exec']
        data = fillorder.append(canorder)
        data = data.sort('microsecond')
        data = data.reset_index()
        data['inventory'] = data['exec buy']-data['exec sell']
        data['inventory'] = data['inventory'].cumsum()
        data['time diff']= data['microsecond'].diff()*1.   
        return data
    
    def HMMPrep(self,df):
        col = ['orderid','cancelled buy','exec sell','cancelled sell','exec buy','microsecond','price','side','time','date','inventory','time diff',
         'ewav_back canc buy','ewav_back canc sell','ewav_back exec buy','ewav_back exec sell','ewav_back buy/sell','ewav_back sell/buy']
        if 'IsSpoof' in df.columns:
            col +=['IsSpoof']
        df = df[col]
        del df['ewav_back exec buy']
        del df['ewav_back exec sell']
        # clean the data for ewav_back canc buy/sell and sell/buy
        # buy/sell will be just inverse of sell/buy, so we use one column buy/sell
        df.loc[(df['ewav_back canc buy']==0)&(df['ewav_back canc sell']==0),'ewav_back buy/sell']=1
        medianbs = df.loc[(df['ewav_back buy/sell']>0)&(df['ewav_back buy/sell']<np.inf),'ewav_back buy/sell'].median()
        maxbs = df.loc[(df['ewav_back buy/sell']>0)&(df['ewav_back buy/sell']<np.inf),'ewav_back buy/sell'].max()
        df.loc[df['ewav_back buy/sell']==np.inf,'ewav_back buy/sell'] = maxbs
        df.loc[df['ewav_back buy/sell']==0,'ewav_back buy/sell'] = 1/maxbs
        df['ewav_back buy/sell'] = df['ewav_back buy/sell'].map(np.log)
        
        ## Get the time difference, seems not contributing for now
        df['TimeDiff_back'] = np.nan
        df['TimeDiff_frwd'] = np.nan
        df['TimeDiff_min'] = np.nan
        df = df.loc[(df['exec sell']>0)|(df['exec buy']>0),:].copy()

        buy = df.loc[df['side']=='B',:].copy()
        for dd in buy['date'].unique():
            #import pdb;pdb.set_trace()
            tmp = buy.loc[buy['date']==dd,:]
            buy.loc[buy['date']==dd,'TimeDiff_back'] = buy.loc[buy['date']==dd,'microsecond'].diff(1).map(lambda x:np.abs(x))
            buy.loc[buy['date']==dd,'TimeDiff_frwd'] = buy.loc[buy['date']==dd,'microsecond'].diff(-1).map(lambda x:np.abs(x))
            #import pdb;pdb.set_trace()    
            buy['TimeDiff_frwd'] = buy['TimeDiff_frwd'].fillna(buy['TimeDiff_frwd'].max())    
            buy['TimeDiff_back'] = buy['TimeDiff_back'].fillna(buy['TimeDiff_back'].max())
            buy['TimeDiff_min'] = buy.apply(lambda x:min(x['TimeDiff_back'],x['TimeDiff_frwd']),axis=1)

        sell = df.loc[df['side']=='S',:].copy()
        for dd in sell['date'].unique():
            tmp = sell.loc[sell['date']==dd,:]
            sell.loc[sell['date']==dd,'TimeDiff_back'] = sell.loc[sell['date']==dd,'microsecond'].diff(1).map(lambda x:np.abs(x))
            sell.loc[sell['date']==dd,'TimeDiff_frwd'] = sell.loc[sell['date']==dd,'microsecond'].diff(-1).map(lambda x:np.abs(x))
    
        sell['TimeDiff_frwd'] = sell['TimeDiff_frwd'].fillna(sell['TimeDiff_frwd'].max())
        sell['TimeDiff_back'] = sell['TimeDiff_back'].fillna(sell['TimeDiff_back'].max())
        sell['TimeDiff_min'] = sell.apply(lambda x:min(x['TimeDiff_back'],x['TimeDiff_frwd']),axis=1)

        newdf = buy.append(sell)
        newdf['date'] = newdf['date'].map(lambda x:pd.to_datetime(x))
        #newdf = newdf.sort(['date','microsecond'])
        df = newdf.sort()
        
        return df


    

In [11]:
class HMM:
    def __init__(self,nState,TDFeaSet,featureSet,useAllFea,useDPGMM=True):
        '''
        recommended value for featureSet=['ewav_back buy/sell']
        '''
        self.TDFeaSet = TDFeaSet
        self.featureSet = featureSet
        self.useDPGMM = useDPGMM
        self.useAllFea = useAllFea
        #self.df = data
        self.nState = nState
        self.tp = None
        self.pi = None
        self.TDmodel = []
        self.RatioModel = []
    
    def DefStates(self,df):
        '''Define states, return data
        '''        
        s1 = {'side':'B','IsSpoof':False}
        s2 = {'side':'S','IsSpoof':False}
        s3 = {'side':'B','IsSpoof':True}
        s4 = {'side':'S','IsSpoof':True}
        df = df.loc[(df['exec sell']>0)|(df['exec buy']>0),:].copy()
        df['state']=0
        df.loc[(df['side']=='B')&(df['IsSpoof']==False),'state'] = 0
        df.loc[(df['side']=='S')&(df['IsSpoof']==False),'state'] = 1
        df.loc[(df['side']=='B')&(df['IsSpoof']==True),'state'] = 2
        df.loc[(df['side']=='S')&(df['IsSpoof']==True),'state'] = 3
        return df
    
    def fitmodels(self,data):
        ''' fits the emission model for each state
        '''
        if self.useDPGMM==True:
            dpgmm = mixture.DPGMM(n_components=5)
        else:
            dpgmm = mixture.GMM(n_components=2)
        #import pdb;pdb.set_trace()
        dpgmm.fit(data)
        return dpgmm
    
    def train(self,df):
        self.pi = np.array(df.groupby('state').size()*1.0/len(df))
        
        df['next state'] = df['state'].shift(-1)    
        xx = pd.DataFrame()
        for dd in df['date'].unique():
            tmp = df.loc[df['date']==dd,:].copy()
            tmp = tmp.reset_index(drop=True)
            tmp = tmp.ix[0:len(tmp)-2]
            if len(tmp)<1:
                continue
            xx = xx.append(tmp)

        gp = xx.groupby(['state','next state','date']).size()
        aa = gp.sum(level=[0,1])
        bb = gp.sum(level=0)*1.
        self.tp = aa/bb

        self.TDmodel = []
        self.RatioModel = []
        for state in range(self.nState):
            #import pdb;pdb.set_trace()
            td = df.loc[df['state']==state,self.TDFeaSet]
            ratio = df.loc[df['state']==state,self.featureSet]
        #m1 = fitmodels(td,isGMM=True)
        #m2 = fitmodels(ratio,isGMM=True)
            m1 = self.fitmodels(td)
            m2 = self.fitmodels(ratio)        
            self.TDmodel.append(m1)
            self.RatioModel.append(m2)
        
    
    def plotDist(self,model):
        xx = np.arange(-10,10,0.25)
        yy = model.score(xx)
        plt.plot(xx,yy)
    
    def plotDist2(self,model1,model2):
        xx = np.arange(-10,10,0.25)
        yy1 = model1.score(xx)
        yy2 = model2.score(xx)
        plt.plot(xx,yy1,'*',xx,yy2,'x')
    
    def stateEstimator(self,obs):
        ''' Estimate the most likely state sequence given the sequence of observations
        The data (obs) should be only one day data
        '''
        nState = self.nState
        TDmodel = self.TDmodel
        RatioModel = self.RatioModel
        tdlist = []
        rtlist = []
    
        for td in TDmodel:
            tdlist.append(td.score(obs[self.TDFeaSet]))
        for rt in RatioModel:    
            rtlist.append(rt.score(obs[self.featureSet]))
    
        tdprob = np.asmatrix(tdlist)
        rtprob = np.asmatrix(rtlist)
        if self.useAllFea == True:
            distrprob = tdprob + rtprob
        else:
            distrprob = rtprob
        logtp = np.log(tp)
        logpi = np.log(pi)
    
        backtrack = np.ones((nState,len(obs)))*(-1)
        pathscore = np.zeros((nState,len(obs)))
        
        isbuy = obs['side'].map(lambda x:int(x=='B'))
        issell = obs['side'].map(lambda x:int(x=='S'))
        validState = np.asmatrix([isbuy,issell,isbuy,issell]) # 0 means not valid
        dumbval = -1e30
    
        ttt = np.squeeze(np.asarray(distrprob[:,0])) + logpi
        pathscore[:,0] = ttt
        for ii in range(nState):
            if validState[ii,0]==0:
                pathscore[ii,0] = dumbval
    
        for ii in range(1,len(obs)):
            for jj in range(nState):
                tmp = logtp[:,jj] + pathscore[:,ii-1]+np.squeeze(np.asarray(distrprob[:,ii]))
                pathscore[jj,ii] = max(tmp)
                backtrack[jj,ii] = np.argmax(tmp)
            for kk in range(nState):
                if validState[kk,ii]==0:
                    pathscore[kk,ii] = dumbval
                    backtrack[kk,ii] = -1
        stateSeq = [-1]*len(obs)
        stateSeq[len(obs)-1] = np.argmax(pathscore[:,len(obs)-1])
        for nn in range(len(obs)-2,-1,-1):
            stateSeq[nn] = backtrack[stateSeq[nn+1],nn+1]
        return stateSeq
    
    def stateProb(self,obs):
        '''Give the estimate of the probablity of each state at each time instance
        '''
        tdlist = []
        rtlist = []
    
        nState = self.nState
        TDmodel = self.TDmodel
        RatioModel = self.RatioModel
        
        for td in TDmodel:
            tdlist.append(td.score(np.array(obs[self.TDFeaSet])))
        for rt in RatioModel:    
            rtlist.append(rt.score(np.array(obs[self.featureSet])))
        
        tdprob = np.asmatrix(tdlist)
        rtprob = np.asmatrix(rtlist)
        if self.useAllFea == True:
            distrprob = tdprob + rtprob
        else:
            distrprob = rtprob        
        logtp = np.log(self.tp)
        logpi = np.log(self.pi)
    
        alpha = np.zeros((nState,len(obs)))
        beta = np.zeros((nState,len(obs)))
    
        isbuy = obs['side'].map(lambda x:int(x=='B'))
        issell = obs['side'].map(lambda x:int(x=='S'))
        validState = np.asmatrix([isbuy,issell,isbuy,issell]) # 0 means not valid
        dumb = -1e5 #used to fill for np.log(zero)
    
        alpha[:,0] = np.squeeze(np.asarray(distrprob[:,0])) + logpi
        for ii in range(1,len(obs)):
            for kk in range(nState):
                if validState[kk,ii]==0:
                    alpha[kk,ii] = dumb
                else:
                    tmp = alpha[:,ii-1] + logtp[:,kk]
                    maxtmp = np.max(tmp)
                    tmp = tmp - maxtmp
                    alpha[kk,ii] = maxtmp + np.log(np.sum(np.exp(tmp))) + distrprob[kk,ii]
        
        for ii in range(len(obs)-2,-1,-1):
            for kk in range(nState):
                if validState[kk,ii] == 0:
                    beta[kk,ii] = dumb
                else:
                    tmp = np.asarray(logtp[kk])+beta[:,ii+1]+np.squeeze(np.asarray(distrprob[:,ii+1]))
                    maxtmp = np.max(tmp)
                    tmp = tmp - maxtmp
                    beta[kk,ii] = maxtmp + np.log(np.sum(np.exp(tmp)))
            
        gamma = alpha+beta # not exactly the gamma
        maxgamma = np.max(gamma,0)
        gamma = gamma - np.kron(np.reshape(maxgamma,(1,len(obs))),np.ones((nState,1)))
        gamma = np.exp(gamma)
        sumgamma = np.kron(np.sum(gamma,0),np.ones((nState,1)))
        gamma = gamma/sumgamma   
        return gamma
    
    def predict(self,df):
        ''' needs more work,better return a dataframe
        '''
        res = pd.DataFrame()
        for xx in df['date'].unique():
            data = df.loc[df['date']==xx,:].copy()
            prob = self.stateProb(data)
            pred = np.argmax(prob,0)
            pred_prob=np.max(prob,0)
            data['pred'] = pred
            data['pred_prob'] = pred_prob
            data['predSpoofing'] = data['pred'].map(lambda x:x>1)
            res = res.append(data)
        return res
    
    def test(self,df):
        all_truth = []
        all_score = []

        all_pred = []
        all_state = []

        for xx in df['date'].unique():
            data = df.loc[df['date']==xx,:].copy()
            prob = self.stateProb(data)
            pred = np.argmax(prob,0)
            pred_prob=np.max(prob,0)
            tmp = (np.array(pred) == np.array(data['state']))
            r = sum(tmp)*1.0/len(tmp)
            truth = map(lambda x:int(x>1),np.array(data['state']))
            score = [y if x>1 else 1-y for x,y in zip(pred,pred_prob)]
            auc = metrics.roc_auc_score(truth,score)
            all_truth = all_truth + truth
            all_score = all_score + score
            all_pred = all_pred + list(pred)
            all_state = all_state + list(data['state'])
    
        auc = metrics.roc_auc_score(all_truth,all_score)
        tmp = (np.array(all_pred) == np.array(all_state))
        rate = sum(tmp)*1./len(tmp)
        return auc,rate
    
   


In [12]:
def CrossValidate(df):
    ''' randomly split the dataset into 70% train, 30% test
            Calculate the AUC and Accuracy
    '''
    dumbHMM = HMM(nState=4,TDFeaSet=['TimeDiff_frwd','TimeDiff_back'],featureSet=['ewav_back buy/sell'],useAllFea=False,useDPGMM=True) #needs this to add state to the data
    df = dumbHMM.DefStates(df)
    alldate = []
    for xx in df['date'].unique():
        tmp = df.loc[df['date']==xx,:]
        if len(tmp)>10:
            alldate.append(xx)
    alldate =  np.array(alldate)

    allrate = []
    allauc = []
    for ii in range(10):
        seq = np.random.choice(len(alldate),int(0.7*len(alldate)),replace=False)
        traindates = alldate[seq]
        traindf = df.loc[df['date'].isin(traindates),:].copy()
        testdf = df.loc[df['date'].isin(traindates)==False,:].copy()
            #mm = HMM(nState=4,TDFeaSet=['TimeDiff_min'],featureSet=['ewav_back buy/sell'],useAllFea=True,useDPGMM=True)
        mm = HMM(nState=4,TDFeaSet=['TimeDiff_frwd','TimeDiff_back'],featureSet=['ewav_back buy/sell'],useAllFea=False,useDPGMM=True)
        mm.train(traindf)
        auc,rate = mm.test(testdf)
        print auc,rate
        allrate.append(rate)
        allauc.append(auc)

    print 'auc={}'.format(np.mean(allauc))
    print 'ratio = {}'.format(np.mean(allrate))

In [13]:
def Predict(df):
    traindf = prepareTrainSet()        
            #mm = HMM(nState=4,TDFeaSet=['TimeDiff_min'],featureSet=['ewav_back buy/sell'],useAllFea=True,useDPGMM=True)
    mm = HMM(nState=4,TDFeaSet=['TimeDiff_frwd','TimeDiff_back'],featureSet=['ewav_back buy/sell'],useAllFea=False,useDPGMM=True)
    mm.train(traindf)
    df = mm.predict(df)
    return df        

In [18]:
#example 1: cross-validation by randomly split the training set into training set and validating set..
df = prepareTrainSet()
np.random.seed(23)
CrossValidate(df)

0.764116777531 0.73595505618
0.728381406573 0.750819672131


c:\Users\caib\Program\Python\Python2_32\lib\site-packages\ipykernel\__main__.py:45: FutureWarning: sort(....) is deprecated, use sort_index(.....)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [8]:
# example 2: to use HMM to make prediction of new data
dp = DataPrep()
allorder,hmmdata = dp.processData('panl.csv')

res = Predict(hmmdata)
tmp = allorder.join(res[['pred','pred_prob','predSpoofing']])
tmp.to_csv('res_panl.csv',index=False)

----Data cleaning----
---- Feature calculation----

C:\Users\caib\Python\Python2-32bit\lib\site-packages\ipykernel\__main__.py:150: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\caib\Python\Python2-32bit\lib\site-packages\ipykernel\__main__.py:179: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\caib\Python\Python2-32bit\lib\site-packages\numpy\lib\function_base.py:3142: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)



----- Prepare for HMM------

C:\Users\caib\Python\Python2-32bit\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\caib\Python\Python2-32bit\lib\site-packages\ipykernel\__main__.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\caib\Python\Python2-32bit\lib\site-packages\ipykernel\__main__.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').